# ファインチューニング
事前に大規模データセットで学習された（pre-trained）モデルの重みを引き継ぎ、目的とするタスク用の新たなデータセットで再学習（微調整）する手法  
少ないデータで高い精度が得られるほか、学習時間や計算資源の節約にもつながる  
### 層の固定と更新
モデルの初期層は汎用的な特徴（エッジや基本パターン）を捉えているため「固定（freeze）」し、最終層やタスクに特化した層だけを再学習させることが一般的
### 小さい学習率
既存の重みを大きく変えないよう、通常の学習よりも小さな学習率で微調整を行う
### タスクに合わせたアーキテクチャの調整
最終層の出力次元を、目的のタスク（分類ならクラス数、回帰なら1など）に合わせて変更する


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
from tqdm import tqdm

# 1. データの準備
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNetは224x224を前提
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 訓練とテストでデータのサブセットを利用（訓練1000サンプル、テスト500サンプル）
train_subset = Subset(trainset, list(range(1000)))
test_subset = Subset(testset, list(range(500)))

trainloader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=2)
testloader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=2)

# 2. 事前学習済みResNet18の読み込み
model = torchvision.models.resnet18(pretrained=True)

# 3. 事前学習済みモデルの重みを固定
for param in model.parameters():
    param.requires_grad = False

# 4. 最終層をタスクに合わせて変更（CIFAR-10は10クラス）
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 5. 微調整するパラメータのみ学習するため、optimizerに渡す
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 6. 学習ループ（進捗表示付き）
epochs = 2
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    # tqdmで進捗バー表示
    for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_subset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")

# 7. 評価
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(testloader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Test Accuracy: {:.4f}".format(correct / total))


Epoch 1/2: 100%|████████████| 32/32 [00:36<00:00,  1.13s/it]


Epoch 1/2, Loss: 2.0362


Epoch 2/2: 100%|████████████| 32/32 [00:35<00:00,  1.12s/it]


Epoch 2/2, Loss: 1.4919


Evaluating: 100%|███████████| 16/16 [00:23<00:00,  1.47s/it]

Test Accuracy: 0.5780
